In [1]:
import pandas as pd

# 엑셀 파일 불러오기 (sheet_name으로 특정 시트 지정 가능)
df_price_raw = pd.read_excel("etf_raw_daily.xlsx", sheet_name='asset_price')

In [2]:
df_price = pd.DataFrame(df_price_raw.iloc[16:,])
df_price = df_price.rename(columns=df_price_raw.iloc[12,1:49])

In [3]:
df_price.set_index('Unnamed: 0', inplace=True)

In [4]:
weekly_day_price = df_price[df_price.index.weekday ==4]

In [5]:
asset_return_weekly = weekly_day_price.pct_change()*100

C:\Users\USER\AppData\Local\Temp\ipykernel_13160\3094994716.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  asset_return_weekly = weekly_day_price.pct_change()*100
C:\Users\USER\AppData\Local\Temp\ipykernel_13160\3094994716.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  asset_return_weekly = weekly_day_price.pct_change()*100


In [6]:
asset_return_weekly.to_csv('asset_return_weekly.csv')

In [53]:
asset_return_weekly

,Stock,Bond,Commodity,DM,EM,US,Europe,Japan,China,Taiwan,...,Intermediate,Low,Diversified,Precious_metals,Energy&Fuel,Industrial_metals,Agriculture,multi_strategy,macro,equity_hedge
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-10,0.645742,-0.087619,-2.983465,0.613596,0.868428,0.942857,0.193627,0.819855,0.439570,-0.569452,...,-0.197363,-0.047203,-0.783149,0.646691,-6.360032,0.794141,1.304877,0.224961,10.361152,NaN
2020-01-17,1.540889,0.055929,-0.745885,1.593610,1.164401,1.968242,1.293124,0.799520,-0.200087,0.188023,...,0.044944,0.082645,-1.071001,0.135951,-0.846883,0.794016,0.354935,0.472183,9.964185,NaN
2020-01-24,-1.027208,0.789718,-4.836177,-0.836262,-2.392682,-1.025643,-0.216695,-0.890469,-3.633103,0.170413,...,1.024259,0.117966,-3.100482,0.698504,-7.430817,-3.859724,-1.083856,-0.446055,-4.576470,NaN
2020-01-31,-2.558914,0.617596,-4.298569,-2.208482,-5.096526,-2.122611,-3.052120,-2.610464,0.000000,-4.903418,...,1.422981,0.294568,-3.180549,0.507234,-4.853294,-4.599956,-2.326990,-0.902862,9.799790,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-03,-0.516662,0.181928,1.930644,-0.475587,-0.892074,-0.475143,0.199140,-0.959804,-5.171024,-1.657323,...,0.293223,0.183352,0.372989,0.820173,4.759207,-2.160945,0.025714,0.458645,4.329124,NaN
2025-01-10,-1.576922,-0.873225,3.131495,-1.586087,-1.501104,-1.942458,0.651331,-1.765003,-1.130810,1.167113,...,-1.104494,-0.158614,4.145817,3.002559,3.528935,2.862073,1.469280,-0.782933,-3.578561,NaN
2025-01-17,2.562780,0.989820,1.222823,2.708647,1.231672,2.910912,2.369501,-1.885513,2.139596,0.759520,...,1.226322,0.305511,1.258679,1.410574,1.710853,2.552059,1.340817,1.341683,10.877958,NaN


# col 1_1 자산별 가격 추이

In [95]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go


df = pd.read_csv("asset_return_weekly.csv", index_col=0, parse_dates=True)

df_col1_1 = df

asset_return_1w = df_col1_1.iloc[-52:,:3]/100

index_df = (1 + asset_return_1w).cumprod() * 100

# **100을 기준으로 지수 계산**
index_df = (1 + asset_return_1w).cumprod() * 100

# **첫 번째 행 추가 (기준값 100)**
start_date = asset_return_1w.index[0] - pd.DateOffset(weeks=1)  # 첫 번째 주 이전 날짜 생성
initial_row = pd.DataFrame([[100, 100, 100]], columns=index_df.columns, index=[start_date])  # 100으로 된 행 추가

# **초기 값(100)과 기존 데이터 결합**
asset_index_df = pd.concat([initial_row, index_df])


In [97]:
asset_index_df.to_csv('asset_index_df.csv')

In [99]:
df_test = pd.read_csv("asset_index_df.csv", index_col=0, parse_dates=True)


In [43]:
import streamlit as st
import pandas as pd
import altair as alt

# 데이터 로드 함수
@st.cache_data
def load_data():
    df = pd.read_csv("asset_return_weekly.csv", index_col=0, parse_dates=True)
    return df

# 데이터 불러오기
df = load_data()
df = round(df, 2)  # 소수점 두 자리 반올림

# 최근 12주 데이터 필터링 (모든 자산 포함)
filtered_df = df.tail(12).T
filtered_df = filtered_df.loc[["Stock", "Bond", "Commodity"], :]  # 순서 유지

# 데이터 변환: Altair를 위한 DataFrame 구조 변경
filtered_df = filtered_df.reset_index().melt(id_vars="index", var_name="Date", value_name="Return")
filtered_df.rename(columns={"index": "Asset"}, inplace=True)

# ✅ "Asset" 컬럼을 Categorical 타입으로 변환하여 순서 유지
asset_order = ["Stock", "Bond", "Commodity"]
filtered_df["Asset"] = pd.Categorical(filtered_df["Asset"], categories=asset_order, ordered=True)

# 날짜 포맷 변경 (YYYY-MM-DD → Jan 15 형식)
filtered_df["Date"] = pd.to_datetime(filtered_df["Date"]).dt.strftime("%b %d")

# ✅ 폰트 크기 및 스타일 설정 (모든 텍스트에 적용)
font_size = 15  # 공통 폰트 크기 설정
font_family = "Times New Roman"  # 원하는 폰트 (예: 'Times New Roman', 'Courier New')

# 히트맵 생성 (색상 조정 + 값 입력 추가)
heatmap = alt.Chart(filtered_df).mark_rect(stroke="black", strokeWidth=0.25).encode(
    x=alt.X("Date:O", title="Date", axis=alt.Axis(labelFontSize=font_size, titleFontSize=font_size, labelFont=font_family, titleFont=font_family)),
    y=alt.Y("Asset:O", title="", sort=asset_order, axis=alt.Axis(labelFontSize=font_size, titleFontSize=font_size, labelFont=font_family, titleFont=font_family)),
    color=alt.Color(
        "Return:Q",
        scale=alt.Scale(domain=[-5, 0, 5], range=["#ffffff", "#c6dbef", "#084594"]),  # 색상 조정
        title="Return"
    ),
    tooltip=["Date", "Asset", "Return"]
).properties(
    width=900,
    height=400
)

# 데이터 라벨 추가 (각 셀에 숫자 표시, 폰트 크기 및 스타일 적용)
text = alt.Chart(filtered_df).mark_text(baseline="middle", fontSize=font_size, font=font_family).encode(
    x="Date:O",
    y=alt.Y("Asset:O", sort=asset_order),
    text=alt.Text("Return:Q", format=".2f"),  # 소수점 2자리 표시
    color=alt.condition(
        alt.datum.Return > 0, alt.value("#4d4d4d"), alt.value("#4d4d4d")  # 밝은 배경이므로 검정색 글씨 유지
    )
)

# ✅ LayerChart에서 폰트 크기 및 스타일 적용
final_chart = (heatmap + text).configure_axis(
    labelFontSize=font_size,
    titleFontSize=font_size,
    labelFont=font_family,  # 축 라벨 폰트 적용
    titleFont=font_family   # 축 제목 폰트 적용
).configure_title(
    fontSize=font_size + 2,
    font=font_family  # 타이틀 폰트 적용
).configure_legend(
    labelFontSize=font_size,
    titleFontSize=font_size,
    labelFont=font_family,  # 범례 폰트 적용
    titleFont=font_family
).configure_text(
    font=font_family  # 히트맵 안 숫자 폰트 적용
)

# 히트맵 출력 (데이터 라벨 포함)
st.altair_chart(final_chart, use_container_width=True)


2025-02-02 15:22:05.282 No runtime found, using MemoryCacheStorageManager


DeltaGenerator()

In [51]:
filtered_df.to_csv('filtered_df.csv')

In [49]:
filtered_df

,Asset,Date,Return
0,Stock,Nov 15,-2.36
1,Bond,Nov 15,-0.84
2,Commodity,Nov 15,-2.09
3,Stock,Nov 22,1.37
4,Bond,Nov 22,0.19
5,Commodity,Nov 22,3.80
6,Stock,Nov 29,0.98
7,Bond,Nov 29,1.39
8,Commodity,Nov 29,-2.10
9,Stock,Dec 06,1.30
